In [42]:
import sys
import os
# Get the current directory and parent directory
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)
sys.path.insert(0, current_dir)

# Import from the-machine module
import importlib.util
the_machine_path = os.path.join(parent_dir, "the-machine")
sys.path.insert(0, the_machine_path)

# Import all from the-machine and custom_matroid_functions
from HM_module import *
from custom_matroid_functions import *

In [43]:
P = LaurentPolynomialRing(ZZ, 'x')
x = P.gen()
k, n = 4, 6
M = matroids.CompleteGraphic(n)
#M = matroids.Uniform(k, n)
L = M.lattice_of_flats()
r = M.rank()
rank_func = lambda x : M.rank(x)

#e = next(x for x in M.groundset() if x  not in M.coloops())
e = list(M.groundset())[0]
print(M.groundset())
print("deleting ", e)
mdele = M.delete(e)
Eq_M = create_deformed_algebra(M, base_ring=P)
Eq_mdele = create_deformed_algebra(mdele, base_ring=P)
E_mdele = Eq_mdele.natural()

E = Eq_M.natural()
E_basis = E.basis()
E_mdele_basis = E_mdele.basis()

C = Eq_M.C()
C_dele = Eq_mdele.C()
C_basis = C.basis()
C_dele_basis = C_dele.basis()

Z = Eq_M.zeta()
Z_dele = Eq_mdele.zeta()
Z_basis = Z.basis()

flats_keys = list(E_basis.keys())

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14})
deleting  0


In [44]:
print("rank of M: ", r)
print("chow polynomial of M: ", chow_polynomial(M))
print(reduced_characteristic_polynomial(M))
C(E(flats_keys[-1]))

rank of M:  5
chow polynomial of M:  x^4 + 187*x^3 + 732*x^2 + 187*x + 1
x^4 - 14*x^3 + 71*x^2 - 154*x + 120


-(15*x+86*x^3+120*x^5)*C[frozenset()] + (10*x^2+24*x^4)*C[frozenset({7})] - (x+6*x^3)*C[frozenset({13, 7})] + 2*x^2*C[frozenset({1, 13, 7})] - x*C[frozenset({1, 6, 7, 8, 12, 13, 14})] - x*C[frozenset({0, 1, 3, 5, 7, 10, 13})] + (10*x^2+24*x^4)*C[frozenset({4})] - x*C[frozenset({1, 2, 4, 7, 9, 11, 13})] + 2*x^2*C[frozenset({6, 7, 8, 12, 13, 14})] + 2*x^2*C[frozenset({9, 11, 13, 7})] - x*C[frozenset({5, 6, 7, 8, 9, 10, 11, 12, 13, 14})] - (x+6*x^3)*C[frozenset({4, 7})] - (x+6*x^3)*C[frozenset({4, 12})] + 2*x^2*C[frozenset({2, 3, 4, 12, 13, 14})] - x*C[frozenset({0, 2, 3, 4, 6, 7, 8, 12, 13, 14})] + 2*x^2*C[frozenset({10, 13, 5, 7})] + 2*x^2*C[frozenset({2, 4, 13, 7})] - x*C[frozenset({2, 4, 5, 7, 10, 13})] + 2*x^2*C[frozenset({0, 3, 13, 7})] - (x+6*x^3)*C[frozenset({11, 7})] - x*C[frozenset({0, 3, 7, 9, 11, 13})] + 2*x^2*C[frozenset({11, 12, 6, 7})] - x*C[frozenset({1, 4, 6, 7, 11, 12})] + 2*x^2*C[frozenset({2, 11, 7})] + 2*x^2*C[frozenset({0, 11, 3, 7})] - x*C[frozenset({0, 2, 3, 6, 7, 

In [45]:
rev = {value: key for key, value in E_basis.items()}

def delta_flat(F):
    F_del_e = frozenset(set(F) - {e})
    return Eq_mdele(x^( - M.rank(F) + mdele.rank(F_del_e)) * E_mdele_basis[F_del_e])
    
def delta(elem):
    rep = E(elem)
    decom = {monomial: coeff for monomial, coeff in zip(rep.monomials(), rep.coefficients())}
    return Eq_mdele(sum(decom[F] * delta_flat(rev[F]) for F in decom))

dict = {F : C_dele(delta(C[F])) for F in E_basis.keys()}

In [46]:
def flat_division(M, e):
    L = M.lattice_of_flats()
    flat_div = {'S':set(), 'T':set(), 'A':set(), 'B':set()}
    for F in L:
        if e in F:
            if F - {e} in L:
                flat_div['T'].add(F)
            else:
                flat_div['A'].add(F)
        else:
            if F.union({e}) in L:
                flat_div['S'].add(F)
            else:
                flat_div['B'].add(F)
    return flat_div

In [47]:
from IPython.display import HTML, display
import pandas as pd

# Get the flat division
flat_div = flat_division(M, e)

# Create separate data lists for each category
data_s = []
data_t = []
data_a = []
data_b = []

basis_list_mdele = list(mdele.lattice_of_flats())

for F in dict:
    elem = dict[F]
    coeff_empty = elem.monomial_coefficients().get(frozenset(), 0)
    
    terms = []
    for S, coeff in elem.monomial_coefficients().items():
        terms.append(f"({coeff})*C[{str(set(S))}]")
    formatted_expr = " + ".join(terms) if terms else "0"
    
    row_data = {
        'F': str(set(F)),
        'rk(F)': M.rank(F),
        'delta(C[F])': formatted_expr,
        'coeff of empty': coeff_empty,
        'in zeta basis': str(Z_dele(elem))
    }
    
    # Categorize the flat and add to appropriate list
    if F in flat_div['S']:
        data_s.append(row_data)
    elif F in flat_div['T']:
        data_t.append(row_data)
    elif F in flat_div['A']:
        data_a.append(row_data)
    elif F in flat_div['B']:
        data_b.append(row_data)

# Create and display tables for each category
def display_table(data, category_name, color='lightblue'):
    if data:
        df = pd.DataFrame(data)
        display(HTML(f'<h3 style="color: {color};">Table for Category {category_name} ({len(data)} flats)</h3>'))
        display(HTML(
            f'<div style="height: 300px; overflow-y: auto; border: 2px solid {color}; padding: 10px;">{df.to_html(index=False)}</div>'
        ))
    else:
        display(HTML(f'<h3 style="color: {color};">Table for Category {category_name} (0 flats)</h3>'))
        display(HTML('<p>No flats in this category.</p>'))

# Display all four tables
display_table(data_s, 'S', 'green')
display_table(data_t, 'T', 'coral') 
display_table(data_a, 'A', 'blue')
display_table(data_b, 'B', 'red')

F,rk(F),delta(C[F]),coeff of empty,in zeta basis
set(),0,(1)*C[set()],1,Z[frozenset()]
{10},1,(1)*C[{10}],0,Z[frozenset({10})]
{13},1,(1)*C[{13}],0,Z[frozenset({13})]
"{10, 13}",2,"(1)*C[{10, 13}]",0,"Z[frozenset({10, 13})]"
{12},1,(1)*C[{12}],0,Z[frozenset({12})]
{14},1,(1)*C[{14}],0,Z[frozenset({14})]
"{12, 13, 14}",2,"(1)*C[{12, 13, 14}]",0,"Z[frozenset({12, 13, 14})]"
{11},1,(1)*C[{11}],0,Z[frozenset({11})]
"{10, 11, 14}",2,"(1)*C[{10, 11, 14}]",0,"Z[frozenset({10, 11, 14})]"
"{11, 12}",2,"(1)*C[{11, 12}]",0,"Z[frozenset({11, 12})]"


F,rk(F),delta(C[F]),coeff of empty,in zeta basis
{0},1,(x^-1 + x)*C[set()],x^-1 + x,(x^-1+x)*Z[frozenset()]
"{0, 10}",2,(x^-1 + x)*C[{10}],0,(x^-1+x)*Z[frozenset({10})]
"{0, 13}",2,(x^-1 + x)*C[{13}],0,(x^-1+x)*Z[frozenset({13})]
"{0, 10, 13}",3,"(x^-1 + x)*C[{10, 13}]",0,"(x^-1+x)*Z[frozenset({10, 13})]"
"{0, 12}",2,(x^-1 + x)*C[{12}],0,(x^-1+x)*Z[frozenset({12})]
"{0, 14}",2,(x^-1 + x)*C[{14}],0,(x^-1+x)*Z[frozenset({14})]
"{0, 12, 13, 14}",3,"(x^-1 + x)*C[{12, 13, 14}]",0,"(x^-1+x)*Z[frozenset({12, 13, 14})]"
"{0, 11}",2,(x^-1 + x)*C[{11}],0,(x^-1+x)*Z[frozenset({11})]
"{0, 10, 11, 14}",3,"(x^-1 + x)*C[{10, 11, 14}]",0,"(x^-1+x)*Z[frozenset({10, 11, 14})]"
"{0, 11, 12}",3,"(x^-1 + x)*C[{11, 12}]",0,"(x^-1+x)*Z[frozenset({11, 12})]"


F,rk(F),delta(C[F]),coeff of empty,in zeta basis
"{0, 1, 5}",2,"(1)*C[set()] + (1)*C[{1, 5}]",1,"Z[frozenset()] + Z[frozenset({1, 5})]"
"{0, 1, 12, 5}",3,"(1)*C[{12}] + (1)*C[{1, 12, 5}]",0,"Z[frozenset({12})] + Z[frozenset({1, 12, 5})]"
"{0, 1, 13, 5}",3,"(1)*C[{13}] + (1)*C[{1, 13, 5}]",0,"Z[frozenset({13})] + Z[frozenset({1, 13, 5})]"
"{0, 2, 6}",2,"(1)*C[set()] + (1)*C[{2, 6}]",1,"Z[frozenset()] + Z[frozenset({2, 6})]"
"{0, 2, 10, 6}",3,"(1)*C[{10}] + (1)*C[{2, 10, 6}]",0,"Z[frozenset({10})] + Z[frozenset({2, 10, 6})]"
"{0, 1, 5, 14}",3,"(1)*C[{14}] + (1)*C[{1, 5, 14}]",0,"Z[frozenset({14})] + Z[frozenset({1, 5, 14})]"
"{0, 2, 6, 14}",3,"(1)*C[{14}] + (1)*C[{2, 6, 14}]",0,"Z[frozenset({14})] + Z[frozenset({2, 6, 14})]"
"{0, 1, 5, 12, 13, 14}",4,"(1)*C[{12, 13, 14}] + (1)*C[{1, 5, 12, 13, 14}]",0,"Z[frozenset({12, 13, 14})] + Z[frozenset({1, 5, 12, 13, 14})]"
"{0, 2, 11, 6}",3,"(1)*C[{11}] + (1)*C[{2, 11, 6}]",0,"Z[frozenset({11})] + Z[frozenset({2, 11, 6})]"
"{0, 2, 6, 10, 11, 14}",4,"(1)*C[{10, 11, 14}] + (1)*C[{2, 6, 10, 11, 14}]",0,"Z[frozenset({10, 11, 14})] + Z[frozenset({2, 6, 10, 11, 14})]"


F,rk(F),delta(C[F]),coeff of empty,in zeta basis
{2},1,(1)*C[{2}],0,Z[frozenset({2})]
"{2, 10}",2,"(1)*C[{2, 10}]",0,"Z[frozenset({2, 10})]"
{3},1,(1)*C[{3}],0,Z[frozenset({3})]
"{3, 13}",2,"(1)*C[{3, 13}]",0,"Z[frozenset({3, 13})]"
{5},1,(1)*C[{5}],0,Z[frozenset({5})]
"{2, 5}",2,"(1)*C[{2, 5}]",0,"Z[frozenset({2, 5})]"
"{3, 5}",2,"(1)*C[{3, 5}]",0,"Z[frozenset({3, 5})]"
"{13, 5}",2,"(1)*C[{13, 5}]",0,"Z[frozenset({13, 5})]"
"{13, 3, 5}",3,"(1)*C[{13, 3, 5}]",0,"Z[frozenset({13, 3, 5})]"
"{2, 3, 12}",2,"(1)*C[{2, 3, 12}]",0,"Z[frozenset({2, 3, 12})]"


In [48]:
#\Delta(C[F]) - C[F\e] = 
def delta_diff(F):
    return delta(C(F)) -C_dele(F - {e})

print(E_basis.keys()[-1])
C_dele(delta_diff(E_basis.keys()[-1]))


frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14})


C[frozenset({10})] + C[frozenset({13})] + C[frozenset({12})] + C[frozenset({14})] + C[frozenset({11})] + C[frozenset({9})] + C[frozenset({9, 10, 11, 12, 13, 14})]

In [58]:
klpoly = lambda M: M.lattice_of_flats().kazhdan_lusztig_polynomial()

def rem_sum(M, f):
    sum = 0
    r = M.rank()
    for F in flat_div['S']:
        sum += P(x^(r-M.rank(F)-2) * f(M.contract(F.union({e})))(x^-2)) * C_dele(F)
    return sum

rem_sum(M, klpoly)

(5*x+x^3)*C[frozenset()] + (1+x^2)*C[frozenset({11})] + (1+x^2)*C[frozenset({14})] + x*C[frozenset({10, 11, 14})] + x*C[frozenset({11, 12})] + (1+x^2)*C[frozenset({9})] + x*C[frozenset({9, 14})] + x*C[frozenset({9, 11, 13})] + (1+x^2)*C[frozenset({10})] + (1+x^2)*C[frozenset({13})] + x*C[frozenset({10, 13})] + (1+x^2)*C[frozenset({12})] + x*C[frozenset({12, 13, 14})] + x*C[frozenset({9, 10, 12})] + C[frozenset({9, 10, 11, 12, 13, 14})]

In [59]:
def delta_diff_comp(M, f):
    ground_set = M.groundset()
    mdele = M.delete(e)    
    sum = 0
    for G in flat_div['S']:
        poly1 = f(M.contract(G))(x^-2)
        poly2 = f(M.contract(G.union({e})))(x^-2)
        poly3 = f(mdele.contract(G))(x^-2)
        poly_coeff = P(x^(M.rank(ground_set) - M.rank(G)) * (poly1 + x^(-2) * poly2 - poly3))
        sum += Eq_mdele(poly_coeff * E_mdele_basis[G])
    return C_dele(sum)

delta_diff_comp(M, reduced_characteristic_polynomial)

(x^-3-16*x^-1+81*x-154*x^3+88*x^5)*C[frozenset()] + (x^-2-9+22*x^2-14*x^4)*C[frozenset({11})] + (x^-2-9+22*x^2-14*x^4)*C[frozenset({14})] + (x^-2-9+22*x^2-14*x^4)*C[frozenset({10})] + (x^-1-4*x+3*x^3)*C[frozenset({10, 11, 14})] + (x^-2-9+22*x^2-14*x^4)*C[frozenset({12})] + (x^-1-4*x+3*x^3)*C[frozenset({11, 12})] + (x^-2-9+22*x^2-14*x^4)*C[frozenset({9})] + (x^-1-4*x+3*x^3)*C[frozenset({9, 14})] + (x^-2-9+22*x^2-14*x^4)*C[frozenset({13})] + (x^-1-4*x+3*x^3)*C[frozenset({9, 11, 13})] + (x^-1-4*x+3*x^3)*C[frozenset({10, 13})] + (x^-1-4*x+3*x^3)*C[frozenset({12, 13, 14})] + (x^-1-4*x+3*x^3)*C[frozenset({9, 10, 12})] + (1-x^2)*C[frozenset({9, 10, 11, 12, 13, 14})]

In [60]:
def delta_diff_comp_2tutte(M, f):
    ground_set = M.groundset()
    mdele = M.delete(e)    
    sum = 0
    for G in flat_div['S']:
        poly = x^(M.rank(ground_set) - M.rank(G)) * (-1+x^-2) * f(M.contract(G.union({e})))(x^-2)
        sum += Eq_mdele(P(poly) * E_mdele_basis[G])
    return C_dele(sum)

delta_diff_comp_2tutte(M, reduced_characteristic_polynomial)

(x^-3-16*x^-1+81*x-154*x^3+88*x^5)*C[frozenset()] + (x^-2-9+22*x^2-14*x^4)*C[frozenset({11})] + (x^-2-9+22*x^2-14*x^4)*C[frozenset({14})] + (x^-2-9+22*x^2-14*x^4)*C[frozenset({10})] + (x^-1-4*x+3*x^3)*C[frozenset({10, 11, 14})] + (x^-2-9+22*x^2-14*x^4)*C[frozenset({12})] + (x^-1-4*x+3*x^3)*C[frozenset({11, 12})] + (x^-2-9+22*x^2-14*x^4)*C[frozenset({9})] + (x^-1-4*x+3*x^3)*C[frozenset({9, 14})] + (x^-2-9+22*x^2-14*x^4)*C[frozenset({13})] + (x^-1-4*x+3*x^3)*C[frozenset({9, 11, 13})] + (x^-1-4*x+3*x^3)*C[frozenset({10, 13})] + (x^-1-4*x+3*x^3)*C[frozenset({12, 13, 14})] + (x^-1-4*x+3*x^3)*C[frozenset({9, 10, 12})] + (1-x^2)*C[frozenset({9, 10, 11, 12, 13, 14})]

In [61]:
def delta_diff_comp_chow(M, f):
    ground_set = M.groundset()
    mdele = M.delete(e)    
    sum = 0
    for G in flat_div['S']:
        poly1 = x^(-M.rank(ground_set) + M.rank(G) + 2)
        poly2 = f(M.contract(G))(x^2)
        poly3 = f(M.contract(G.union({e})))(x^2)
        poly4 = f(mdele.contract(G))(x^2)
        poly_coeff = P(poly1 * (poly2 + poly3 - poly4))
        term = C_dele(poly_coeff * E_mdele_basis[G])
        sum += term
    return C_dele(sum)

diff = delta_diff_comp_chow(M, chow_polynomial)
print(diff)

(x^-3+49*x^-1+49*x+x^3)*C[frozenset()] + (x^-2+9+x^2)*C[frozenset({9})] + (x^-1+x)*C[frozenset({9, 10, 12})] + (x^-1+x)*C[frozenset({9, 14})] + (x^-2+9+x^2)*C[frozenset({11})] + (x^-1+x)*C[frozenset({9, 11, 13})] + (x^-2+9+x^2)*C[frozenset({12})] + (x^-1+x)*C[frozenset({11, 12})] + (x^-2+9+x^2)*C[frozenset({14})] + (x^-1+x)*C[frozenset({10, 11, 14})] + (x^-2+9+x^2)*C[frozenset({13})] + (x^-1+x)*C[frozenset({12, 13, 14})] + (x^-2+9+x^2)*C[frozenset({10})] + (x^-1+x)*C[frozenset({10, 13})] + C[frozenset({9, 10, 11, 12, 13, 14})]


In [62]:
def delta_diff_chow_conj(M, f):
    ground_set = M.groundset()  
    sum = 0
    for G in flat_div['S']:
        poly1 = x^(M.rank(ground_set) - M.rank(G) - 2)
        poly2 = f(M.contract(G.union({e})))(x^-2)
        sum += Eq_mdele(P(poly1 * poly2) * C_dele_basis[G])
    return C_dele(sum)

delta_diff_chow_conj(M, chow_polynomial)


(x^-3+41*x^-1+41*x+x^3)*C[frozenset()] + (x^-2+8+x^2)*C[frozenset({11})] + (x^-2+8+x^2)*C[frozenset({14})] + (x^-1+x)*C[frozenset({10, 11, 14})] + (x^-1+x)*C[frozenset({11, 12})] + (x^-2+8+x^2)*C[frozenset({9})] + (x^-1+x)*C[frozenset({9, 14})] + (x^-1+x)*C[frozenset({9, 11, 13})] + (x^-2+8+x^2)*C[frozenset({10})] + (x^-2+8+x^2)*C[frozenset({13})] + (x^-1+x)*C[frozenset({10, 13})] + (x^-2+8+x^2)*C[frozenset({12})] + (x^-1+x)*C[frozenset({12, 13, 14})] + (x^-1+x)*C[frozenset({9, 10, 12})] + C[frozenset({9, 10, 11, 12, 13, 14})]

In [63]:
delta_diff_chow_conj(M, chow_polynomial) == delta_diff(flats_keys[-1])

False

In [64]:
chi = reduced_characteristic_polynomial
flat_div['S']

{frozenset(),
 frozenset({11}),
 frozenset({14}),
 frozenset({10, 11, 14}),
 frozenset({11, 12}),
 frozenset({9}),
 frozenset({9, 14}),
 frozenset({9, 11, 13}),
 frozenset({10}),
 frozenset({13}),
 frozenset({10, 13}),
 frozenset({12}),
 frozenset({12, 13, 14}),
 frozenset({9, 10, 12}),
 frozenset({9, 10, 11, 12, 13, 14})}

In [65]:
def delta_diff_chow_conj_2(M, f):
    ground_set = M.groundset()
    mdele = M.delete(e)
    sum = 0
    for F in flat_div['S']:
        print("iterating over", F)
        print("G in L.order_ideal([F]):", L.order_ideal([F]))
        res = M.delete(ground_set - set(F))
        sumF = 0
        for G in L.order_ideal([F]):
            chibar = chi(res.contract(G))(x^-2)
            poly1 = x^(M.rank(ground_set) - M.rank(G))
            poly2 = f(M.contract(F))(x^-2)*chibar
            poly3 = f(M.contract(F.union({e})))(x^-2)*chibar
            poly4 = f(mdele.contract(F))(x^-2)*chibar
            term = P(poly1 * (poly2 + x^(-2)*poly3 - poly4))
            print(G, term)
            sumF += - C_dele(term * C_dele_basis[G])
        print("summing over", F, "gives", sumF)
        sum += sumF
    return sum

delta_diff_chow_conj_2(M, chow_polynomial)

iterating over frozenset({10, 11, 14})
G in L.order_ideal([F]): [frozenset(), frozenset({10}), frozenset({14}), frozenset({11}), frozenset({10, 11, 14})]
frozenset() x^-1 - 4*x^3
frozenset({10}) 1 + 2*x^2
frozenset({14}) 1 + 2*x^2
frozenset({11}) 1 + 2*x^2
frozenset({10, 11, 14}) -x^-1 - 2*x
summing over frozenset({10, 11, 14}) gives -(x^-1-4*x^3)*C[frozenset()] - (1+2*x^2)*C[frozenset({10})] - (1+2*x^2)*C[frozenset({14})] - (1+2*x^2)*C[frozenset({11})] + (x^-1+2*x)*C[frozenset({10, 11, 14})]
iterating over frozenset({11})
G in L.order_ideal([F]): [frozenset(), frozenset({11})]
frozenset() x^-1 + 12*x + 5*x^3
frozenset({11}) -x^-2 - 12 - 5*x^2
summing over frozenset({11}) gives -(x^-1+12*x+5*x^3)*C[frozenset()] + (x^-2+12+5*x^2)*C[frozenset({11})]
iterating over frozenset({11, 12})
G in L.order_ideal([F]): [frozenset(), frozenset({12}), frozenset({11}), frozenset({11, 12})]
frozenset() x^-1 + x - 2*x^3
frozenset({12}) 1 + 2*x^2
frozenset({11}) 1 + 2*x^2
frozenset({11, 12}) -x^-1 - 2*x


(x^-3+41*x^-1+41*x+x^3)*C[frozenset()] + (x^-2+8+x^2)*C[frozenset({10})] + (x^-2+8+x^2)*C[frozenset({13})] + (x^-1+x)*C[frozenset({10, 13})] + (x^-2+8+x^2)*C[frozenset({12})] + (x^-2+8+x^2)*C[frozenset({14})] + (x^-1+x)*C[frozenset({12, 13, 14})] + (x^-2+8+x^2)*C[frozenset({11})] + (x^-1+x)*C[frozenset({10, 11, 14})] + (x^-1+x)*C[frozenset({11, 12})] + (x^-2+8+x^2)*C[frozenset({9})] + (x^-1+x)*C[frozenset({9, 11, 13})] + (x^-1+x)*C[frozenset({9, 14})] + (x^-1+x)*C[frozenset({9, 10, 12})] + C[frozenset({9, 10, 11, 12, 13, 14})]

In [66]:
def delta_diff_chow_conj_fixG(M, G, f):
    print("G:", G)
    ground_set = M.groundset()
    mdele = M.delete(e)
    sum = 0
    Fs = set(L.order_filter([G])).intersection(flat_div['S'])
    print("Fs:", Fs)
    for F in Fs:
        minor = M.delete(ground_set - set(F)).contract(G)
        chibar = chi(minor)(x^-2)
        poly1 = f(M.contract(F))(x^-2)
        poly2 = f(M.contract(F.union({e})))(x^-2)
        poly3 = f(mdele.contract(F))(x^-2)
        term = -P((poly1 + x^(-2)*poly2 - poly3)*chibar * x^(M.rank(ground_set) - M.rank(G)))
        print("flat", F, "gives:\n")
        print("f(M.contract(F))(x^-2) =", poly1)
        print("f(M.contract(F.union({e})))(x^-2) =", poly2)
        print("f(mdele.contract(F))(x^-2) =", poly3)
        print("chibar =", chibar)
        print("term =", term, "\n")
        sum += term
    
    assert x^(M.rank(ground_set) - M.rank(G) - 2)*f(M.contract(G.union({e})))(x^-2) == sum
    return sum

for G in flat_div['S']:
    print("--------------------------------")
    print(G,": ", delta_diff_chow_conj_fixG(M, G, chow_polynomial))
    print("--------------------------------")
    print("\n")


--------------------------------
G: frozenset({10, 11, 14})
Fs: {frozenset({10, 11, 14}), frozenset({9, 10, 11, 12, 13, 14})}
flat frozenset({10, 11, 14}) gives:

f(M.contract(F))(x^-2) = x^-4 + 8*x^-2 + 1
f(M.contract(F.union({e})))(x^-2) = x^-2 + 1
f(mdele.contract(F))(x^-2) = x^-4 + 7*x^-2 + 1
chibar = -1
term = x^-1 + 2*x 

flat frozenset({9, 10, 11, 12, 13, 14}) gives:

f(M.contract(F))(x^-2) = x^-2 + 1
f(M.contract(F.union({e})))(x^-2) = 1
f(mdele.contract(F))(x^-2) = x^-2 + 1
chibar = 1
term = -x 

frozenset({10, 11, 14}) :  x^-1 + x
--------------------------------


--------------------------------
G: frozenset({11})
Fs: {frozenset({10, 11, 14}), frozenset({11}), frozenset({11, 12}), frozenset({9, 11, 13}), frozenset({9, 10, 11, 12, 13, 14})}
flat frozenset({10, 11, 14}) gives:

f(M.contract(F))(x^-2) = x^-4 + 8*x^-2 + 1
f(M.contract(F.union({e})))(x^-2) = x^-2 + 1
f(mdele.contract(F))(x^-2) = x^-4 + 7*x^-2 + 1
chibar = 1
term = -1 - 2*x^2 

flat frozenset({11}) gives:

f(M.co